In [1]:
import cv2
import xml.etree.ElementTree as ET
import os
from pascal_voc_io import PascalVocWriter

In [2]:
DIR_BASE = '/extHDD1/16'
FACE_ANNOTATION_DIR = 'face_xml'
PERSON_ANNOTATION_DIR = 'person_xml'
MERGED_ANNOTATION_DIR = 'face_person_xml'
IMAGE_DIR = 'images'

In [3]:
face_annotations = os.listdir(os.path.join(DIR_BASE, FACE_ANNOTATION_DIR))
person_annotations = os.listdir(os.path.join(DIR_BASE, PERSON_ANNOTATION_DIR))
images = os.listdir(os.path.join(DIR_BASE, IMAGE_DIR))

face_annotations.sort()
person_annotations.sort()
images.sort()

In [4]:
def load_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    objects = []
    boxes = []
    size = None
    
    for element in root:
        if element.tag == 'size':
            width = int(element.find('width').text)
            height = int(element.find('height').text)
            size = (width, height)
        if element.tag == 'object':
            n_ele = element.find('name').text
            if n_ele == None: 
                continue
            
            bndbox = element.find('bndbox')
            xmin = convert_string_to_int(bndbox.find('xmin').text)
            ymin = convert_string_to_int(bndbox.find('ymin').text)
            xmax = convert_string_to_int(bndbox.find('xmax').text)
            ymax = convert_string_to_int(bndbox.find('ymax').text)
            
            objects.append(n_ele)
            boxes.append((xmin, ymin, xmax, ymax))
        
    return size, objects, boxes

def create_shapes(boxes, labels):
    assert len(boxes) == len(labels), "Length of boxes and labels must equal."
    shapes = []
    for i in range(len(boxes)):
        label = labels[i]
        box = boxes[i]
        shapes.append((box, label))
    return shapes

def save_pascal_voc_format(filename, shapes, image_path, image_w, image_h):
#     img_folder_path = os.path.dirname(image_path)
#     img_folder_name = os.path.split(img_folder_path)[-1]
#     img_file_name = os.path.basename(image_path)
    #imgFileNameWithoutExt = os.path.splitext(imgFileName)[0]
    # Read from file path because self.imageData might be empty if saving to
    # Pascal format
    image_shape = [image_h, image_w]
    writer = PascalVocWriter("", "",
                             image_shape, localImgPath="")

    for shape in shapes:
#         print(shape)
        difficult = 0
        bndbox, label = shape
        xmin = bndbox[0]
        ymin = bndbox[1]
        xmax = bndbox[2]
        ymax = bndbox[3]
        writer.addBndBox(xmin, ymin, xmax, ymax, label, difficult)
    print('file name: ', filename)
    print('image shape: ', image_shape)
    writer.save(targetFile=filename)
    
def convert_string_to_int(s):
    if '.' in s:
        return int(float(s))
    else:
        return int(s)

In [ ]:
name_list = person_annotations
if len(face_annotations) > len(person_annotations):
    name_list = face_annotations

for name in name_list:
    
    if name not in face_annotations:
        person_anno = os.path.join(DIR_BASE, PERSON_ANNOTATION_DIR, name)
        size, p_obj, p_boxes = load_xml(person_anno)
        print('size: ', size)
        shapes = []
        if len(p_boxes) > 0:
            shapes += create_shapes(p_boxes, p_obj)
    else:
        face_anno = os.path.join(DIR_BASE, FACE_ANNOTATION_DIR, name)
        person_anno = os.path.join(DIR_BASE, PERSON_ANNOTATION_DIR, name)
    
        size, p_obj, p_boxes = load_xml(person_anno)
        _, f_obj, f_boxes = load_xml(face_anno)
        print('size: ', size)
        shapes = []
        if len(p_boxes) > 0:
            shapes += create_shapes(p_boxes, p_obj)
        if len(f_boxes) > 0:
            shapes += create_shapes(f_boxes, f_obj)
            
    print(shapes)
    if size is None:
        image_path = os.path.join(DIR_BASE, IMAGE_DIR, name).replace('.xml', '.jpg')
        size = cv2.imread(image_path).shape[:2][::-1]
        print('size: ', size)
    save_pascal_voc_format(os.path.join(DIR_BASE, MERGED_ANNOTATION_DIR, name), shapes, "", size[0], size[1])